In [20]:
import yfinance as yf
import pandas as pd 
from sec_cik_mapper import StockMapper
from dataclasses import dataclass
from datetime import datetime, timedelta
import logging 
from scipy.optimize import minimize
import numpy as np

# Setup logging
logging.basicConfig(level=logging.INFO)

# Define the month codes used in futures tickers (e.g., F for January, G for February)
months = {
    "F": "Jan", "G": "Feb", "H": "Mar", "J": "Apr",
    "K": "May", "M": "Jun", "N": "Jul", "Q": "Aug",
    "U": "Sep", "V": "Oct", "X": "Nov", "Z": "Dec"
}

#---------------------------------------------------------
# Functions
#---------------------------------------------------------

# Function to retrieve historical data for a single commodity
def get_commodity_data(ticker, start_date, end_date):
    """
    Retrieves historical data on prices for a given commodity.

    Args:
        ticker (str): The commodity ticker
        start_date (str): Start date in the format 'YYYY-MM-DD'
        end_date (str): End date in the format 'YYYY-MM-DD'

    Returns:
        pd.DataFrame: A pandas dataframe with the historical data

    Example:
        df = get_commodity_data('CL=F', '2020-01-01', '2020-12-31')
    """
    try:
        commodity = yf.Ticker(ticker)
        data = commodity.history(start=start_date, end=end_date, auto_adjust=False, actions=False)
        df = pd.DataFrame(data)
        df['ticker'] = ticker
        df.reset_index(inplace=True)
        return df
    except Exception as e:
        logging.warning(f"Error retrieving data for {ticker}: {e}")
        return pd.DataFrame()

# Function to retrieve historical data for multiple commodities
def get_commodities_data(tickers, start_date, end_date):
    """
    Retrieves historical data on prices for a list of commodities.

    Args:
        tickers (list): List of commodity tickers
        start_date (str): Start date in the format 'YYYY-MM-DD'
        end_date (str): End date in the format 'YYYY-MM-DD'

    Returns:
        pd.DataFrame: A pandas dataframe with the historical data

    Example:
        df = get_commodities_data(['CL=F', 'NG=F'], '2020-01-01', '2020-12-31')
    """
    dfs = []
    for ticker in tickers:
        try:
            df = get_commodity_data(ticker, start_date, end_date)
            if not df.empty:
                dfs.append(df)
        except Exception as e:
            logging.warning(f"Commodity {ticker} not found: {e}")
    # Concatenate all dataframes
    data = pd.concat(dfs, ignore_index=True)
    return data

#---------------------------------------------------------
# Example Usage
#---------------------------------------------------------

# Example commodity tickers
commodity_tickers = ['CL=F']  # Crude Oil and Natural Gas

# Get historical data for commodities
start_date = '2000-01-01'
end_date = '2023-12-31'

commodities_data = get_commodities_data(commodity_tickers, start_date, end_date)
print(commodities_data)

#---------------------------------------------------------
# Commodities tickers
#---------------------------------------------------------

# Soft Commodities


                          Date       Open       High        Low      Close  \
0    2000-08-23 00:00:00-04:00  31.950001  32.799999  31.950001  32.049999   
1    2000-08-24 00:00:00-04:00  31.900000  32.240002  31.400000  31.629999   
2    2000-08-25 00:00:00-04:00  31.700001  32.099998  31.320000  32.049999   
3    2000-08-28 00:00:00-04:00  32.040001  32.919998  31.860001  32.869999   
4    2000-08-29 00:00:00-04:00  32.820000  33.029999  32.560001  32.720001   
...                        ...        ...        ...        ...        ...   
5858 2023-12-22 00:00:00-05:00  73.910004  74.980003  73.389999  73.559998   
5859 2023-12-26 00:00:00-05:00  73.559998  76.180000  73.129997  75.570000   
5860 2023-12-27 00:00:00-05:00  75.320000  75.660004  73.769997  74.110001   
5861 2023-12-28 00:00:00-05:00  73.800003  74.400002  71.720001  71.769997   
5862 2023-12-29 00:00:00-05:00  71.989998  72.620003  71.250000  71.650002   

      Adj Close  Volume ticker  
0     32.049999   79385   CL=F

In [23]:
import datetime

# Timestamp from the data
expire_date_timestamp = 1737417600

# Convert to a readable date
expire_date = datetime.datetime.utcfromtimestamp(expire_date_timestamp)
print(f"Expiration Date: {expire_date}")

Expiration Date: 2025-01-21 00:00:00


In [33]:
import yfinance as yf
from datetime import datetime

def get_expiry_date_from_yfinance(ticker):
    """
    Fetch the expiry date of a futures contract from Yahoo Finance.

    Args:
        ticker (str): The ticker symbol of the contract (e.g., "CL=F").

    Returns:
        datetime or None: Expiry date if available, None otherwise.
    """
    try:
        # Initialize the Ticker object
        contract = yf.Ticker(ticker)
        
        # Retrieve contract information
        info = contract.info
        
        # Extract the expiry date if available
        if 'expireDate' in info:
            expiry_timestamp = info['expireDate']
            expiry_date = datetime.utcfromtimestamp(expiry_timestamp)  # Corrected usage
            return expiry_date
        else:
            print(f"No expiry date found for {ticker}.")
            return None
    except Exception as e:
        print(f"Error retrieving expiry date for {ticker}: {e}")
        return None

# Example usage
ticker = "ZS=F"
expiry_date = get_expiry_date_from_yfinance(ticker)
if expiry_date:
    print(f"The expiry date for {ticker} is {expiry_date.date()}.")
else:
    print(f"No expiry date available for {ticker}.")


The expiry date for ZS=F is 2025-03-14.
